In [1]:
import mysql.connector

import pandas as pd

In [63]:
# dataBase Connection
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="Cross_Market_Analysis"   # make sure DB exists
)

cursor = conn.cursor()

if conn.is_connected():
    print("Connected to MySQL successfully")

Connected to MySQL successfully


In [5]:
# Create DataBase Cross_Market_Analysis
cursor.execute("CREATE DATABASE IF NOT EXISTS Cross_Market_Analysis")
conn.database = "Cross_Market_Analysis"

In [19]:
# Create table for cryptoCurrency
cursor.execute("""
CREATE TABLE IF NOT EXISTS cryptocurrencies (
    id VARCHAR(100) PRIMARY KEY,
    symbol VARCHAR(10),
    name VARCHAR(100),
    current_price DECIMAL(18,6),
    market_cap BIGINT,
    market_cap_rank INT,
    total_volume BIGINT,
    circulating_supply DECIMAL(30,6),
    total_supply DECIMAL(30,6),
    ath DECIMAL(18,6),
    atl DECIMAL(18,6),
    date DATE
)
""")

In [3]:
# Read file from CSV file
df = pd.read_csv("D:\\Python_Code\\Project1-Cross-MarketAnalysisCryptoOil&StocksWithSQL&Streamlit\\coingecko_crypto_metadata.csv")


In [4]:
df.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,8057865.00,161009045005654,1,1.610090e+14,3.678040e+12,8147186.00,...,1.998112e+07,2.100000e+07,11187013.00,-27.97126,2025-10-06T18:57:42.558Z,3993.420000,2.016786e+05,2013-07-05T00:00:00.000Z,NaN,2026-01-27T12:37:20.137Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,267712.00,32308386619540,2,3.230839e+13,2.285062e+12,270545.00,...,1.206944e+08,NaN,431946.00,-38.02176,2025-08-24T19:21:03.333Z,28.130000,9.515548e+05,2015-10-20T00:00:00.000Z,"{'times': 43.42287550315072, 'currency': 'btc'...",2026-01-27T12:37:19.462Z
2,tether,usdt,Tether,https://coin-images.coingecko.com/coins/images...,91.52,17071591379746,3,1.757192e+13,6.735900e+12,91.78,...,1.920023e+11,NaN,105.52,-13.22015,2025-01-25T16:51:57.989Z,36.860000,1.484265e+02,2015-03-02T00:00:00.000Z,NaN,2026-01-27T12:37:12.146Z
3,binancecoin,bnb,BNB,https://coin-images.coingecko.com/coins/images...,80682.00,11002755952152,4,1.100276e+13,1.097506e+11,81346.00,...,1.363604e+08,2.000000e+08,121422.00,-33.55221,2025-10-13T08:41:24.131Z,2.580000,3.121386e+06,2017-10-19T00:00:00.000Z,NaN,2026-01-27T12:37:19.801Z
4,ripple,xrp,XRP,https://coin-images.coingecko.com/coins/images...,172.84,10521403776143,5,1.728733e+13,2.206304e+11,177.47,...,9.998572e+10,1.000000e+11,313.99,-44.95373,2025-07-21T15:02:00.541Z,0.159343,1.083693e+05,2013-08-16T00:00:00.000Z,NaN,2026-01-27T12:37:13.303Z


In [5]:
# Data Cleaning - take the columns which are required from the dataFrame
df_clean = df[[
    "id",
    "symbol",
    "name",
    "current_price",
    "market_cap",
    "market_cap_rank",
    "total_volume",
    "circulating_supply",
    "total_supply",
    "ath",
    "atl",
    "last_updated"
]]

In [ ]:
# Convert the date to default - yyyy-mm-dd
df_clean["date"] = pd.to_datetime(df_clean["last_updated"]).dt.date
df_clean.drop(columns=["last_updated"], inplace=True)

In [12]:
# Handle NaN values
df_clean.fillna(0, inplace=True)

# Remove duplicate IDs
df_clean.drop_duplicates(subset=["id"], inplace=True)

C:\Users\bless\AppData\Local\Temp\ipykernel_24176\795705647.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.fillna(0, inplace=True)
C:\Users\bless\AppData\Local\Temp\ipykernel_24176\795705647.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop_duplicates(subset=["id"], inplace=True)


In [10]:
df_clean.head()

,id,symbol,name,current_price,market_cap,market_cap_rank,total_volume,circulating_supply,total_supply,ath,atl,date
0,bitcoin,btc,Bitcoin,8057865.00,161009045005654,1,3.678040e+12,1.998112e+07,1.998112e+07,11187013.00,3993.420000,2026-01-27
1,ethereum,eth,Ethereum,267712.00,32308386619540,2,2.285062e+12,1.206944e+08,1.206944e+08,431946.00,28.130000,2026-01-27
2,tether,usdt,Tether,91.52,17071591379746,3,6.735900e+12,1.865354e+11,1.920023e+11,105.52,36.860000,2026-01-27
3,binancecoin,bnb,BNB,80682.00,11002755952152,4,1.097506e+11,1.363604e+08,1.363604e+08,121422.00,2.580000,2026-01-27
4,ripple,xrp,XRP,172.84,10521403776143,5,2.206304e+11,6.085323e+10,9.998572e+10,313.99,0.159343,2026-01-27


In [25]:
# Query to insert the data from dataFrame to dataBase
query = """
INSERT INTO cryptocurrencies
(id, symbol, name, current_price, market_cap, market_cap_rank,
 total_volume, circulating_supply, total_supply, ath, atl, date)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

data = df_clean.values.tolist()

try:
    cursor.executemany(query, data)
    conn.commit()
    if cursor.rowcount > 0:
        print("✅ Data inserted successfully")
    else:
        print("⚠️ No data inserted")
except mysql.connector.Error as e:
    print("MySQL Error:", e)

finally:
    cursor.close()
    conn.close()

MySQL Error: 1062 (23000): Duplicate entry 'bitcoin' for key 'cryptocurrencies.PRIMARY'


In [ ]:
# Create CryptoPrices table
cursor.execute("""CREATE TABLE crypto_prices (
    coin_id VARCHAR(50),
    date DATE,
    price_usd DECIMAL(18,6),
    PRIMARY KEY (coin_id, date),
    CONSTRAINT fk_crypto
        FOREIGN KEY (coin_id)
        REFERENCES cryptocurrencies(id)
)""")

In [31]:
# load the data from the csv file to the dataFrame
df_crypto_prices = pd.read_csv("top3_coins_historical_prices.csv")

# Convert date format DD-MM-YYYY → DATE
df_crypto_prices["date"] = pd.to_datetime(df_crypto_prices["date"]).dt.date

# Handle NaN
#df_crypto_prices = df_crypto_prices.where(pd.notnull(df), None)

print(df_crypto_prices.head())

   coin_id        date     price_usd
0  bitcoin  2025-01-28  8.813005e+06
1  bitcoin  2025-01-29  8.773589e+06
2  bitcoin  2025-01-30  8.986343e+06
3  bitcoin  2025-01-31  9.079031e+06
4  bitcoin  2025-02-01  8.876282e+06


In [32]:
# select the required columns
df_crypto_prices_clean = df_crypto_prices[[
    "coin_id",
    "date",
    "price_usd"
]]

In [34]:
# Handle NaN values
df_crypto_prices_clean.fillna(0, inplace=True)

# Remove duplicate IDs
df_crypto_prices_clean.drop_duplicates(subset=["coin_id","date"], inplace=True)

In [35]:
# Query to insert the data from dataFrame to dataBase
query = """
INSERT INTO crypto_prices
(coin_id, date, price_usd)
VALUES (%s, %s, %s)
"""

data = df_crypto_prices_clean.values.tolist()

try:
    cursor.executemany(query, data)
    conn.commit()
    if cursor.rowcount > 0:
        print("✅ Data inserted successfully")
    else:
        print("⚠️ No data inserted")
except mysql.connector.Error as e:
    print("MySQL Error:", e)

finally:
    cursor.close()
    conn.close()

✅ Data inserted successfully


In [38]:
# Create table for Oil_Prices
cursor.execute("""CREATE TABLE oil_prices (
    date DATE PRIMARY KEY,
    price_usd DECIMAL(18,6))
""")

In [44]:
# Load the data from the csv file to the dataFrame
df_oil_prices = pd.read_csv("wti_oil_prices_Jan_2020_to_Jan_2026.csv")

# Convert date format DD-MM-YYYY → DATE
df_oil_prices["Date"] = pd.to_datetime(df_oil_prices["Date"]).dt.date

print(df_oil_prices.head())

         Date  wti_price_usd
0  2020-01-02          61.17
1  2020-01-03          63.00
2  2020-01-06          63.27
3  2020-01-07          62.70
4  2020-01-08          59.65


In [45]:
# Cleaning the data and select the required fields
df_oil_prices_clean = df_oil_prices_clean[[
    "date",
    "price_usd"
]]

NameError: name 'df_oil_prices_clean' is not defined

In [46]:
# Handle NaN values
df_oil_prices.fillna(0, inplace=True)

# Remove duplicate IDs
df_oil_prices.drop_duplicates(subset=["Date","wti_price_usd"], inplace=True)

In [47]:
# Query to insert the data from DataFrame to OilPrice tabel in DB
query = """
INSERT INTO oil_prices
(date, price_usd)
VALUES (%s, %s)
"""

data = df_oil_prices.values.tolist()

try:
    cursor.executemany(query, data)
    conn.commit()
    if cursor.rowcount > 0:
        print("✅ Data inserted successfully")
    else:
        print("⚠️ No data inserted")
except mysql.connector.Error as e:
    print("MySQL Error:", e)

finally:
    cursor.close()
    conn.close()

✅ Data inserted successfully


In [50]:
# Create table for stock_prices
cursor.execute("""CREATE TABLE stock_prices (
    date DATE,
    open DECIMAL(18,6),
    high DECIMAL(18,6),
    low DECIMAL(18,6),
    close DECIMAL(18,6),
    volume BIGINT,
    ticker VARCHAR(20),
    PRIMARY KEY (date, ticker)
)""")

In [51]:
# Load the data from the csv file to dataFrame
df_stock_prices = pd.read_csv("stock_indices_daily_2020_2025.csv")

# Convert date format DD-MM-YYYY → DATE
df_stock_prices["date"] = pd.to_datetime(df_stock_prices["date"]).dt.date

print(df_stock_prices.head())

  ticker        date         open         high          low        close  \
0  ^GSPC  2020-01-02  3244.669922  3258.139893  3235.530029  3257.850098   
1  ^GSPC  2020-01-03  3226.360107  3246.149902  3222.340088  3234.850098   
2  ^GSPC  2020-01-06  3217.550049  3246.840088  3214.639893  3246.280029   
3  ^GSPC  2020-01-07  3241.860107  3244.909912  3232.429932  3237.179932   
4  ^GSPC  2020-01-08  3238.590088  3267.070068  3236.669922  3253.050049   

     adj_close      volume  
0  3257.850098  3459930000  
1  3234.850098  3484700000  
2  3246.280029  3702460000  
3  3237.179932  3435910000  
4  3253.050049  3726840000  


In [53]:
# Clean the data and select the required fields
df_stock_prices_clean = df_stock_prices[[
    "date",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "ticker"
]]

In [54]:
# Handle NaN values
df_stock_prices_clean.fillna(0, inplace=True)

# Remove duplicate IDs
df_stock_prices_clean.drop_duplicates(subset=["date","ticker"], inplace=True)

C:\Users\bless\AppData\Local\Temp\ipykernel_24176\1601388821.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock_prices_clean.fillna(0, inplace=True)
C:\Users\bless\AppData\Local\Temp\ipykernel_24176\1601388821.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock_prices_clean.drop_duplicates(subset=["date","ticker"], inplace=True)


In [64]:
# Query to insert the data from DataFrame to stock_price table in the DB
query = """
INSERT INTO stock_prices
(date, open, high, low, close, volume, ticker)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

data = df_stock_prices_clean.values.tolist()

try:
    cursor.executemany(query, data)
    conn.commit()
    if cursor.rowcount > 0:
        print("✅ Data inserted successfully")
    else:
        print("⚠️ No data inserted")
except mysql.connector.Error as e:
    print("MySQL Error:", e)

finally:
    cursor.close()
    conn.close()

✅ Data inserted successfully
